# Plot Infrastructure Maps
This notebook creates maps of new power plant sitings and retirements under each scenario.

## imports

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
from shapely import Point
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

import os

### Settings

In [2]:
# list of scenarios for analysis
scenario_list = ['net_zero_ira_ccs_climate', 'business_as_usual_ira_ccs_climate']

# list of years for analysis
year_list = list(range(2025, 2050+1, 5))

west_states = ['washington', 'oregon', 'california', 'idaho', 'montana', 'new_mexico', 'arizona', 'utah', 'nevada', 'colorado', 'wyoming']


# coordinates and labels for specified cities
city_label_data = {
    'city': ['Albuquerque', 'San Francisco', 'Los Angeles', 'Portland', 'Seattle', 'Phoenix', 'Salt Lake City', 'Denver', 'Boise',
              "Billings", "Cheyenne", "Las Vegas"], 
    'Latitude': [35.0853336,  37.7749295, 34.0522342, 45.5230622, 47.6062095, 33.4483771, 40.7607793, 39.7392358, 43.6187102, 
                  45.7832856, 41.1399814,  36.1699412],  
    'Longitude': [-106.6055534, -122.4194155,  -118.2436849, -122.6764816, -122.3320708,-112.0740373,-111.8910474, -104.990251, -116.2146068, 
                   -108.5006904,-104.8202462, -115.1398296]}

retired_color = '#631200'

color_map_dict={
    'Natural Gas':'#bdb59d',
    'Coal': '#75736d',
    'Solar PV': '#F7B615',
    'Solar CSP': '#f1781b',
    'Wind': '#95C060', 
    'Nuclear': '#6D547C', 
    'Offshore Wind':'#2B7F9E', 
    'Biomass': '#713275',
    'Geothermal': '#BB3B1C', 
    'Oil': '#BB3B1C',
    'Hydro': '#14345e'}

markersize = 8

# create map legend elements
natgas = Line2D([0], [0], lw=0, markersize=markersize, label='Natural Gas',marker='o', 
                markerfacecolor=color_map_dict['Natural Gas'], markeredgecolor=color_map_dict['Natural Gas'])
solar_pv = Line2D([0], [0], lw=0, markersize=markersize, label='Solar PV',marker='o', 
                  markerfacecolor=color_map_dict['Solar PV'], markeredgecolor=color_map_dict['Solar PV'])
solar_csp = Line2D([0], [0], lw=0, markersize=markersize, label='Solar CSP',marker='o', 
                   markerfacecolor=color_map_dict['Solar CSP'], markeredgecolor=color_map_dict['Solar CSP'])
wind = Line2D([0], [0], lw=0, markersize=markersize,label='Wind',marker='o', 
              markerfacecolor=color_map_dict['Wind'], markeredgecolor=color_map_dict['Wind'])
wind_off = Line2D([0], [0], lw=0, markersize=markersize, label='Offshore Wind',marker='o', 
                  markerfacecolor=color_map_dict['Offshore Wind'], markeredgecolor=color_map_dict['Offshore Wind'])
coal= Line2D([0], [0], lw=0, markersize=markersize, label='Coal',marker='o', 
             markerfacecolor=color_map_dict['Coal'], markeredgecolor=color_map_dict['Coal'])
biomass = Line2D([0], [0], lw=0, markersize=markersize, label='Biomass',marker='o', 
                 markerfacecolor=color_map_dict['Biomass'], markeredgecolor=color_map_dict['Biomass'])
geothermal = Line2D([0], [0], lw=0, markersize=markersize, label='Geothermal',marker='o', 
                    markerfacecolor=color_map_dict['Geothermal'], markeredgecolor=color_map_dict['Geothermal'])
hydro = Line2D([0], [0], lw=0, markersize=markersize, label='Hydro',marker='o', 
               markerfacecolor=color_map_dict['Hydro'], markeredgecolor=color_map_dict['Hydro'])

retired = Line2D([0], [0], lw=0, markersize=markersize, label='Retired Power Plants',marker='x', color=retired_color)
volt = Line2D([0], [0], color='black', lw=1, label='Transmission >250kV')

legend_elements = [coal, natgas, biomass, geothermal,solar_csp, solar_pv, wind, wind_off, hydro, retired, volt]

short_legend_elements = [coal, natgas, biomass, solar_csp, solar_pv, wind, wind_off, retired, volt]
short_legend_no_ret_elements = [coal, natgas, biomass, solar_csp, solar_pv, wind, wind_off, volt]

### Collect Data Paths

In [3]:
# data dir
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'input_data')

# output figure dir
fig_dir = os.path.join(os.path.dirname(os.getcwd()), 'figures', 'infrastructure_maps')

# western interconnection shapefile
wecc_shp_path =  os.path.join(data_dir, "wecc_boundary_shp", "wecc_boundary.shp")

# western interconnection transmission shapefile
transmission_path = os.path.join(data_dir, 'transmission_shp', 'wecc_transmission_250kv.shp')

# infrastructure siting output csv
infrastucture_path = os.path.join(data_dir, 'infrastructure_data_csv', 'infrastructure_data_2050.csv')

### Functions

In [4]:
def results_to_geodataframe(df, crs = "ESRI:102003"):
    """ 
    Takes a pandas DataFrame with x and y coordinates as input and 
    converts to a GeoPandas GeodataFrame. Coordinates in DataFrame are expected to
    follow the ESRI:102003 albers equal area conic coordinate referece system.
    x-coordinate column should be called 'xcoord', y-coordinate column should be
    called 'ycoord'

    :param df:        input Pandas DataFrame with x/y coordinates
    :type df:         Pandas DataFrame
    
    :param crs:       Coordinate reference system to use for GeoDataFrame
    :type crs:        str
    
    """
    
    # create geometry column from coordinate fields
    geometry = [Point(xy) for xy in zip(df['xcoord'], df['ycoord'])]
    
    gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return gdf


def plot_map(gdf, scenario, city_gdf, year, title, output_file_name, all_operational=False, short_leg=False):
    """
    Maps power plant sitings for given scenario and run year. All CERF-sited power plants by
    given run year will be sited along with all power plants that were retired up to the same year.
    If the all_operational parameter is set to True, this function will also plot all power plants
    that are operational in the given year, both CERF-sited and non CERF-sited (i.e., pre-existing infrastructure.
    The resulting map is save as a png file to the figures folder.

    :param gdf:                       Geopandas GeoDataFrame of power plants (new cerf sited, non-cerf sited operational, and non-cerf sited retired)
    :type gdf:                        geopandas.GeoDataFrame
    
    :param scenario:                  Name of scenario to plot
    :type scenario:                   str

    :param city_gdf:                  Geopandas GeoDataFrame of major city locations
    :type city_gdf:                   geopandas.GeoDataFrame

    :param year:                      Year that values should be plotted for
    :type year:                       int  

    :param title:                     Plot title
    :type title:                      str  

    :param output_file_name:          Name of file to save output image as
    :type output_file_name:           str  

    :param all_operational:           If True, all operational plants are plotted. If False, only new infrastructure and retired plants are plotted.
    :type all_operational:            bool     

    """
    
    # set figure size
    fig, ax = plt.subplots(figsize=(18,20))
    ax.set_axis_off();

    # plot wecc boundary
    wecc_shp.boundary.plot(ax=ax, color = 'black')

    # plot cerf-sited plants
    gdf_cerf = gdf[(gdf.cerf_sited == 1) & (gdf.scenario == scenario)]
    gdf_cerf.plot(ax=ax, markersize=12,  marker = 'o', lw=.3, facecolor=gdf_cerf['tech_name_simple'].map(color_map_dict))

    # plot retired plants
    gdf[(gdf.retirement_year <= year) & (gdf.scenario == scenario)].plot(ax=ax, markersize=25, color=retired_color, marker="x")

    if all_operational:
        gdf_op = gdf[(gdf.retirement_year > year) & (gdf.scenario == scenario) & (gdf.cerf_sited == 0)].copy()
        gdf_op.plot(ax=ax, markersize=12,  marker = 'o', lw=.3, facecolor=gdf_op['tech_name_simple'].map(color_map_dict))
    else:
        pass
    
    # plot transmission line
    transmission_shp.plot(ax=ax, color='black', lw=.75)

    # plot city locations
    city_gdf.plot(ax=ax, color='black')

    # add city labels
    for x, y, label in zip(city_gdf.geometry.x, city_gdf.geometry.y, city_gdf.city):
        ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points", ha='right', size=12)

    if short_leg:
        plt.legend(handles=short_legend_elements, fontsize=16, title_fontsize = 16, loc='upper right', ncol=1, facecolor='white')
        
    else:
        # add legend
        plt.legend(handles=legend_elements, fontsize=16, title_fontsize = 16, loc='upper right', ncol=1, facecolor='white')

    # add title
    plt.title(title, size=20)

    plt.tight_layout()

    # save figure to folder
    plt.savefig(os.path.join(fig_dir, output_file_name))
    
    return plt.show()


### Collect Data

In [5]:
# read in wecc shapefile
wecc_shp = gpd.read_file(wecc_shp_path)

# read in wecc transmission shapefile
transmission_shp = gpd.read_file(transmission_path)

# collect infrastructure data
df = pd.read_csv(infrastucture_path)

### Create Geodataframes

In [6]:
year = 2050

# create geodataframe from infrastucture data
gdf = results_to_geodataframe(df=df)
gdf = gdf[gdf.region_name.isin(west_states)]
gdf = gdf[gdf.sited_year <= year]

# create a geodataframe of major cities to plot
city_df = pd.DataFrame.from_dict(city_label_data)
city_gdf = gpd.GeoDataFrame(city_df, geometry=gpd.points_from_xy(city_df.Longitude, city_df.Latitude), crs="EPSG:4326")
city_gdf.to_crs('ESRI:102003', inplace=True)

## Create Maps

### Figure 2 - Map of New Infrastructure and Retirements by Year for Net Zero Scenario

In [ ]:
scenario = 'net_zero_ira_ccs_climate'
gdf_plot = gdf[gdf.scenario == scenario]

height = 16
width = 11

fig = plt.figure()
fig.set_figheight(height)
fig.set_figwidth(width)

spec = GridSpec(ncols=2, 
                nrows=4,
                width_ratios=[.5, .5], 
                wspace=0,
                hspace=0,#-.080, 
                height_ratios=[.3, .3, .3, .06])


ax1a =fig.add_subplot(spec[0,0])
ax1b =fig.add_subplot(spec[0,1])
ax2a =fig.add_subplot(spec[1,0])
ax2b =fig.add_subplot(spec[1,1])
ax3a =fig.add_subplot(spec[2,0])
ax3b =fig.add_subplot(spec[2,1])
ax_legend = fig.add_subplot(spec[3,:])

ax_objs = [ax1a, ax1b, ax2a, ax2b, ax3a, ax3b]
gdf_cerf = gdf_plot[(gdf_plot.cerf_sited == 1)]
gdf_all = gdf_plot.copy()
sited_year_list = sorted(list(gdf_cerf.sited_year.unique()))

i=0
for a in ax_objs:
    wecc_shp.boundary.plot(ax=a, color = 'black', lw=.75)

     # plot retired plants
    gdf_ret_plot = gdf_all[gdf_all.retirement_year <= sited_year_list[i]]
    gdf_ret_plot.plot(ax=a, markersize=1, color=retired_color, marker="x")

    
    gdf_plot = gdf_cerf[(gdf_cerf.sited_year <= sited_year_list[i]) & (gdf_cerf.retirement_year > sited_year_list[i]) ]
    gdf_plot.plot(ax=a, markersize=1,  marker = 'o', lw=.3, facecolor=gdf_plot['tech_name_simple'].map(color_map_dict))

   
    
    a.annotate(sited_year_list[i], xy=(-2.44e6, 1.45e6),size=14)
    
    transmission_shp.plot(ax=a, color='black', lw=.25)
    city_gdf.plot(ax=a, color='black', markersize=3)
    
    # add city labels
    for x, y, label in zip(city_gdf.geometry.x, city_gdf.geometry.y, city_gdf.city):
        a.annotate(label, xy=(x, y), xytext=(3, -3), textcoords="offset points", ha='right', size=7)
    i+=1
    a.set_axis_off()

ax_legend.legend(handles=short_legend_elements, fontsize=10, loc='center', ncol=5, facecolor='white')
ax_legend.annotate('Note: All natural gas and coal-fired generation deployed past 2035 includes carbon capture sequestration' , 
                   xy=(0.05, .02), ha='left', size=8)
ax_legend.set_axis_off()
plt.tight_layout()

 # save figure to folder
output_file_name = 'fig_2_net_zero_sitings_by_year_2050.png'
plt.savefig(os.path.join(fig_dir, output_file_name), dpi=700)

plt.show()

### Additional Figures

In [ ]:
scenario = 'business_as_usual_ira_ccs_climate'
gdf_plot = gdf[gdf.scenario == scenario]

height = 16
width = 11

fig = plt.figure()
fig.set_figheight(height)
fig.set_figwidth(width)

spec = GridSpec(ncols=2, 
                nrows=4,
                width_ratios=[.5, .5], 
                wspace=0,
                hspace=0,
                height_ratios=[.3, .3, .3, .06])


ax1a =fig.add_subplot(spec[0,0])
ax1b =fig.add_subplot(spec[0,1])
ax2a =fig.add_subplot(spec[1,0])
ax2b =fig.add_subplot(spec[1,1])
ax3a =fig.add_subplot(spec[2,0])
ax3b =fig.add_subplot(spec[2,1])
ax_legend = fig.add_subplot(spec[3,:])

ax_objs = [ax1a, ax1b, ax2a, ax2b, ax3a, ax3b]
gdf_cerf = gdf_plot[(gdf_plot.cerf_sited == 1)]
gdf_all = gdf_plot.copy()
sited_year_list = sorted(list(gdf_cerf.sited_year.unique()))

i=0
for a in ax_objs:
    wecc_shp.boundary.plot(ax=a, color = 'black', lw=.75)

     # plot retired plants
    gdf_ret_plot = gdf_all[gdf_all.retirement_year <= sited_year_list[i]]
    gdf_ret_plot.plot(ax=a, markersize=1, color=retired_color, marker="x")

    
    gdf_plot = gdf_cerf[(gdf_cerf.sited_year <= sited_year_list[i]) & (gdf_cerf.retirement_year > sited_year_list[i]) ]
    gdf_plot.plot(ax=a, markersize=1,  marker = 'o', lw=.3, facecolor=gdf_plot['tech_name_simple'].map(color_map_dict))

   
    
    a.annotate(sited_year_list[i], xy=(-2.44e6, 1.45e6),size=14)
    
    transmission_shp.plot(ax=a, color='black', lw=.25)
    city_gdf.plot(ax=a, color='black', markersize=3)
    
    # add city labels
    for x, y, label in zip(city_gdf.geometry.x, city_gdf.geometry.y, city_gdf.city):
        a.annotate(label, xy=(x, y), xytext=(3, -3), textcoords="offset points", ha='right', size=7)
    i+=1
    a.set_axis_off()

ax_legend.legend(handles=short_legend_elements, fontsize=10, loc='center', ncol=5, facecolor='white')
ax_legend.annotate('Note: All natural gas and coal-fired generation deployed past 2035 includes carbon capture sequestration' , 
                   xy=(0.05, .02), ha='left', size=8)
ax_legend.set_axis_off()
plt.tight_layout()

 # save figure to folder
output_file_name = 'bau_sitings_by_year_2050.png'
plt.savefig(os.path.join(fig_dir, output_file_name), dpi=700)

plt.show()

#### Net Zero - New Power Plant Sitings and Retirements, 2050

In [ ]:
plot_map(gdf=gdf, scenario='net_zero_ira_ccs_climate', city_gdf=city_gdf, year=year,
         output_file_name = f'map_net_zero_{year}.png',
         all_operational=False, 
        short_leg=True,
         title = 'Net Zero, New Power Plant Sitings and Retirements by 2050')

#### Business-as-usual - New Power Plant Sitings and Retirements

In [ ]:
plot_map(gdf=gdf, scenario='business_as_usual_ira_ccs_climate', city_gdf=city_gdf, year=year,
         output_file_name = f'map_bau_{year}.png',
         all_operational=False, 
         title = f'Business-as-usual, New Power Plant Sitings and Retirements by 2050')